In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import skrf as rf
from tqdm.notebook import tqdm

import sys; sys.path.append('..')
from west_ic_antenna import WestIcrhAntenna
from west_ic_antenna.topica import TopicaResult

# Plasma Coupling Using TOPICA Results

In this example, we use the the TOPICA front-face coupling calculations which have been used during the design of the WEST ICRH Antenna. TOPICA results are typically given as Z-matrix, which here have been converted into S-parameter files to be imported in the antenna RF model. 

TOPICA results are given for single frequency (55 MHz). If the antenna is defined for a frequency band, the S-parameters of the TOPICA front-face are then duplicated behind the hood for all the frequencies of the frequency band. 

The following function generates a scikit-rf Network object, interpolated from TOPICA results obtained during the design of the WEST ICRH Antenna. These Network that can be used later as the input front-face of the antenna.  

In [ ]:
# generates a 2 Ohm coupling resistance front-face
WestIcrhAntenna.interpolate_front_face(Rc=2, source='TOPICA-L-mode')

In the example below, we build a WEST ICRH antenna model from a TOPICA-interpolated result and we match the antenna

In [ ]:
# Antenna Frequency band defined as single point
f0 = 55e6
freq = rf.Frequency(f0, f0, npoints=1, unit='Hz')
# generate antenna front face
TOPICA_front_face = WestIcrhAntenna.interpolate_front_face(Rc=2, source='TOPICA-L-mode')
# build the antenna model
antenna = WestIcrhAntenna(frequency=freq, front_face=TOPICA_front_face)
# match the antenna in dipole
power = [1, 1]
phase = [0, np.pi]

C_match = antenna.match_both_sides(f_match=f0, power=power, phase=phase)

In [ ]:
C_match

In [ ]:
_Cs = C_match.copy()
_Cs[0] += 2
_Cs[1] -= 2
_Cs[2] += 2
_Cs[3] -= 2

for idx in np.arange(30):
    _Cs_left, _Cs_right, _ = antenna.capacitor_predictor(power, phase, Cs=list(_Cs), K=0.3 )
    _Cs = np.array([_Cs_left, _Cs_right]).reshape(4)
    print(_Cs, '->', antenna.vswr_act(power, phase, Cs=list(_Cs)))

In [ ]:
C_match

In [ ]:
 antenna.capacitor_predictor(power, phase, Cs=list(_Cs) )

In this example, the antenna is :

 1. matched for the low coupling case, 

 2. then the antenna is be presented to higher coupling situation 

it aims to demonstrate the load tolerance properties of the antenna. This scenario is somewhat similar to the rise of an ELM in front of the antenna. 

In [ ]:
# dipole excitation
power = [1, 1]
phase = [0, np.pi] 

# create a reference case
ff = WestIcrhAntenna.interpolate_front_face(Rc=2, source='TOPICA-L-mode')
antenna = WestIcrhAntenna(frequency=freq, front_face=ff)

# tune the antenna
antenna.C_match =  antenna.match_both_sides()

Now that the antenna is tuned let's vary the coupling case and check the resulting VSWR and coupling resistance Rc

In [ ]:
freq = rf.Frequency(54, 56, npoints=1001, unit='MHz')

Rcs_ideal, Rcs, VSWR = [], [], []

Rcs_target = np.arange(start=1, stop=2.9, step=0.2)
for Rc in tqdm(Rcs_target):
    ff = WestIcrhAntenna.interpolate_front_face(Rc=Rc, source='TOPICA-L-mode')
    antenna = WestIcrhAntenna(frequency=freq, front_face=ff)
    antenna.Cs = C_match  # use match point calculated previously
    
    # plasma ideal coupling resistance
    Rcs_ideal.append(antenna.front_face_Rc()[freq.center_idx,0])
    # achieved antenna coupling resistance
    Rcs.append(antenna.Rc_WEST(power, phase)[freq.center_idx,:])
    # VSWR
    VSWR.append(antenna.vswr_act(power, phase)[freq.center_idx])

Rcs_ideal = np.array(Rcs_ideal)
Rcs = np.array(Rcs)
VSWR = np.array(VSWR)

In [ ]:
fig, ax = plt.subplots()
ax.plot(Rcs[:,0], VSWR[:,0], '.', label='left')
ax.plot(Rcs[:,1], VSWR[:,1], 'x', label='right')
ax.set_ylim(bottom=1)
ax.legend()

In [ ]:
from IPython.core.display import HTML
def _set_css_style(css_file_path):
    """
    Read the custom CSS file and load it into Jupyter
    Pass the file path to the CSS file
    """
    styles = open(css_file_path, "r").read()
    s = '<style>%s</style>' % styles
    return HTML(s)

_set_css_style('custom.css')